There are three video folders, video_0 and video_1 is for training purpose and video_2 is for test.csv. We have to output a file similar to example_sample_submission.csv

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
print("ok")

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
x = [{'x': 559, 'y': 213, 'width': 50, 'height': 32}]
print(len(x))

Just to see what the file contains

In [ ]:
train_file = open("/kaggle/input/tensorflow-great-barrier-reef/train.csv").read()
train_file

Reading the training data file

In [ ]:
train = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
train

It may seems like that annotations is empty but there are fields below which have values as shown below. Empty annotation means that there is no coral reefs in it.

Example of image where annotation is not empty, this means that the image has a coral reefs in it.

In [ ]:
train.iloc[17]

In [ ]:
test = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/test.csv")
test

Loading test examples, that we have to execute on our system.

In [ ]:
example_test = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/example_sample_submission.csv")
example_test

In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics


In [ ]:
pip install opencv-python

In [ ]:
pip install --upgrade ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import PIL 
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import os 
import pathlib 

# Sample run before traning

In [ ]:
model = YOLO("yolov8m.pt") # medium size I have chosen

In [ ]:
# results = model.predict()
results=model.predict(source="/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/10925.jpg",
              save=True, conf=.25)

# conf: object confidence threshold for detection
#Iou: intersection over union (IoU) threshold for NMS
result = results[0]
# Plotting results
res_plotted = results[0].plot()
res_plotted = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
display(Image.fromarray(res_plotted))

**As expected nothing was detected, as we need to train the model on our data before model could detect correct data.**

In [ ]:
len(result.boxes)

**This should give error as no object was detected**, we will run this example again after the traning of model to visualize the difference.

In [ ]:
box = result.boxes[0]

In [ ]:
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    probs = result.probs  # Class probabilities for classification outputs

In [ ]:
boxes

# initialization

In [ ]:
!pip install opencv-python
!pip install ultralytics
!pip install --upgrade ultralytics
!pip install --upgrade ultralytics ray

In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import PIL 
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import os 
import pathlib 

# **Preparing Data**

dataset/
├── images/
│   ├── train/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   └── val/
│       ├── image101.jpg
│       ├── image102.jpg
│       └── ...
└── labels/
    ├── train/
    │   ├── image1.txt
    │   ├── image2.txt
    │   └── ...
    └── val/
        ├── image101.txt
        ├── image102.txt
        └── ...


In [ ]:
import pandas as pd
import os

# Load the CSV
train = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")

# Specify directories
images_dir = "/kaggle/working/images/train/"
labels_dir = "/kaggle/working/labels/train/"
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
print("ok")

In [ ]:
# Define function to convert bounding box format
def convert_to_yolo_format(row, image_width, image_height):
    annotations = eval(row['annotations'])  # Convert string representation to list
    yolo_annotations = []
    
    for annotation in annotations:
        x = annotation['x']
        y = annotation['y']
        width = annotation['width']
        height = annotation['height']
        
        # Calculate YOLO format
        x_center = (x + width / 2) / image_width
        y_center = (y + height / 2) / image_height
        width /= image_width
        height /= image_height
        
        yolo_annotations.append(f"0 {x_center} {y_center} {width} {height}")
    
    return yolo_annotations


In [ ]:
# Process each row to create label files
for _, row in train.iterrows():
    image_id = row['image_id']
    annotations = row['annotations']
    # Check if there are annotations
    if annotations == "[]":
        continue  # Skip images with no annotations

    # Set image dimensions (adjust this if known)
    image_width = 1280
    image_height = 720
    
    # Convert annotations to YOLO format
    yolo_annotations = convert_to_yolo_format(row, image_width, image_height)
    # Write YOLO annotations to label file
    label_path = os.path.join(labels_dir, f"{image_id}.txt")
    with open(label_path, 'w') as f:
        f.write("\n".join(yolo_annotations))

print("ok")

Using Symlinks

In [ ]:
import os
from glob import glob

# Define source directories and target directory
source_dir = "/kaggle/input/tensorflow-great-barrier-reef/train_images/"
train_images_dir = "/kaggle/working/images/train/"
os.makedirs(train_images_dir, exist_ok=True)

# Create symlinks for each image from video folders to the train directory
for video_folder in ["video_0", "video_1"]:
    # Extract the folder ID (e.g., "0" from "video_0")
    folder_id = video_folder.split('_')[1]
    video_path = os.path.join(source_dir, video_folder)
    images = glob(os.path.join(video_path, "*.jpg"))
    
    for img_path in images:
        img_name = os.path.basename(img_path)
        # Create the new symlink name as folder_id + "-" + img_name
        new_img_name = f"{folder_id}-{img_name}"
        symlink_path = os.path.join(train_images_dir, new_img_name)
        
        # Create a symlink instead of copying
        if not os.path.exists(symlink_path):  # Avoid overwriting if it already exists
            os.symlink(img_path, symlink_path)
print("ok")

In [ ]:
import yaml

# Define the configuration as a dictionary
data_config = {
    'path': '/kaggle/working/',
    'train': 'images/train',
    'val': 'images/train',  # Ensure this is correct if you have validation images
    'names': ['coral']  # List of class names, replace 'coral' with your actual class names if needed
}

# Write the dictionary to a YAML file
with open('/kaggle/working/data.yaml', 'w') as file:
    yaml.dump(data_config, file)

print("data.yaml created successfully at /kaggle/working/data.yaml")

# Verify the contents of data.yaml
with open('/kaggle/working/data.yaml', 'r') as file:
    data_yaml_content = file.read()
    print(data_yaml_content)


**Checking Total Number Of Label Files**

In [ ]:
import os

# Path to the labels directory
labels_dir = "/kaggle/working/labels/train"

# Check if any label files exist and print a few examples
label_files = os.listdir(labels_dir)
print(f"Number of label files: {len(label_files)}")

# Print the contents of a sample label file, if it exists
if label_files:
    sample_label_path = os.path.join(labels_dir, label_files[0])
    with open(sample_label_path, 'r') as f:
        print(f"Contents of {sample_label_path}:\n", f.read())
else:
    print("No label files found.")


# **Training begins**

# YOLO MEDIUM

Printing the models

In [ ]:
print("*************************small***********************")
s_model = YOLO('yolov8s.pt')
print(s_model.model)

print("*************************medium***********************")
m_model = YOLO('yolov8m.pt')
print(m_model.model)

print("*************************large***********************")
l_model = YOLO('yolov8l.pt')
print(l_model.model)

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8s.pt')

# Training The Final Model
Result_Final_model = Final_model.train(data="/kaggle/working/data.yaml", seed=0, epochs = 5, batch = -1, optimizer = 'auto')

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(data="/kaggle/working/data.yaml", seed=0, epochs = 5, batch = -1, optimizer = 'auto')

# YOLO Large

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8l.pt')

# Training The Final Model
Result_Final_model = Final_model.train(data="/kaggle/working/data.yaml", seed=0, epochs = 5, batch = -1, optimizer = 'auto')

**YOLO Medium is giving better performance out of the box for some reason.**

S,M,L WITH SPECIFIC PARAMETERS

In [ ]:
# Build from YAML and transfer weights
S_model = YOLO('yolov8s.pt')

# Training The Final Model
S_model = S_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # Use AdamW for improved convergence
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

Proceeding with yolo medium

# Base Case Medium Model going forward

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # Use AdamW for improved convergence
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

# SGD

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # CHANGED
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

# Base case SGD going forward

removed warmup ephocs

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    # removed warmup epochs
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

# Base case Augmentation

In [ ]:
import yaml

# Define the configuration as a dictionary
data_config = {
    'path': '/kaggle/working/',
    'train': 'images/train',
    'val': 'images/train',  # Ensure this is correct if you have validation images
    'names': ['coral'],  # List of class names, replace 'coral' with your actual class names if needed
    'augment': {
        'flipud': 0.5,           # Flip images vertically with a probability of 50%
        'fliplr': 0.5,           # Flip images horizontally with a probability of 50%
        'mosaic': 0.8,           # Use mosaic augmentation 80% of the time
        'mixup': 0.2,            # Use mixup augmentation 20% of the time
        'degrees': 10.0,         # Range of random rotation in degrees
        'translate': 0.1,        # Range of random translation, as a fraction of the image size
        'scale': 0.5,            # Range of random scaling, as a fraction of the image size
        'shear': 2.0,            # Range of random shear in degrees
        'perspective': 0.0001,   # Perspective change range for transforming the image
        'hsv_h': 0.015,          # Hue adjustment range in the HSV color space
        'hsv_s': 0.7,            # Saturation adjustment range in the HSV color space
        'hsv_v': 0.4,            # Brightness adjustment range in the HSV color space
        'brightness': 0.2,       # Adjust brightness by ±20%
        'contrast': 0.2,         # Adjust contrast by ±20%
        'blur': 0.2,             # Probability of applying a blur effect
        'noise': 0.1,            # Probability of adding Gaussian noise
        'sharpness': 0.3,        # Range of sharpness adjustment
        'motion_blur': 0.1,      # Probability of applying motion blur
        'grayscale': 0.1,        # Probability of converting the image to grayscale
        'cutout': 0.5,           # Probability of applying cutout augmentation
        'cutmix': 0.3,           # Probability of applying cutmix augmentation
        'mosaic_ratio': 0.4      # Specifies how much the mosaic affects the combined images
    }
}

# Write the dictionary to a YAML file
with open('/kaggle/working/data.yaml', 'w') as file:
    yaml.dump(data_config, file)

print("data.yaml created successfully at /kaggle/working/data.yaml")

# Verify the contents of data.yaml
with open('/kaggle/working/data.yaml', 'r') as file:
    data_yaml_content = file.read()
    print(data_yaml_content)


In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # CHANGED
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

augment=True,                       # Enable basic augmentations
    mosaic=0.8,                         # Mosaic probability
    mixup=0.2                           # Mixup probability

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # CHANGED
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization

    augment=True,                       # Enable basic augmentations
    mosaic=0.8,                         # Mosaic probability
    mixup=0.2                           # Mixup probability
)

More augmentations

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # CHANGED
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization

    augment=True,                       # Enable basic augmentations
    
    # Augmentation parameters
    hsv_h=0.015,                        # Hue shift (0.0 - 0.5), where 0.015 is a small shift
    hsv_s=0.7,                          # Saturation shift (0.0 - 0.9)
    hsv_v=0.4,                          # Brightness shift (0.0 - 0.9)
    degrees=10.0,                       # Range of rotation in degrees (-180 to 180)
    translate=0.1,                      # Range of translation (0.0 - 1.0), fraction of image size
    scale=0.5,                          # Scale factor range (0.0 - 1.0)
    shear=2.0,                          # Shear angle in degrees
    perspective=0.0001,                 # Perspective distortion (0.0 - 0.001)
    flipud=0.5,                         # Probability of vertical flip (0.0 - 1.0)
    fliplr=0.5,                         # Probability of horizontal flip (0.0 - 1.0)
    mosaic=1.0,                         # Probability of using mosaic augmentation (0.0 - 1.0)
    mixup=0.2,                          # Probability of using mixup augmentation (0.0 - 1.0)
    copy_paste=0.0,                     # Probability of copy-paste augmentation (0.0 - 1.0)
    auto_augment='randaugment',         # Auto augment policy ('randaugment', 'trivialaugment', etc.)
    erasing=0.4,                        # Probability of random erasing (cutout) augmentation
    crop_fraction=1.0 
)

# Normalization

It is already part of yolo v8 model

# Regularization

**DROPOUT**
This would require me to alter the underlying model of the yolo, and dropout works best when data is less compared to the model size
But still test out the droput=0.1

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    dropout=0.1,#added
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    dropout=0.01,#added
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

# Testing various parameters other than Augmentation, Optimizer, ephocs, batch size

change the Confidence value, lower or make it hight maybe
conf=0.3

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    conf=0.3 #higher, default is .25
)

conf=0.2

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    conf=0.2 #lower, default is .2
)

# Learning Rates

lr0=0.005

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    conf=0.25               #default is .2
)

lr0=0.01

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.01, # Higher, earlier 0.002 then it was .005
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.0005,     # Regularization
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    conf=0.25               #default is .2
)

# Weight Decay

weight_decay=0.001

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.001, # Increased, initial 0.0005
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

weight_decay=0.01,

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.01, # Increased further, initial 0.0005 then 0.001
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

weight_decay=0.05

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # SGD
    lr0=0.002,               # Slightly higher initial learning rate than the large model
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.05, # Increased further, initial 0.0005 then 0.001 then .01
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True              # Cosine learning rate decay for smoother optimization
)

# Learning rate schedules

While Cosine Decay is the default and works well in YOLOv8, other schedules may be better suited for specific datasets, tasks, or training requirements. For example:

Use OneCycleLR or Warmup for large models or when initializing from scratch.
Use Step Decay for tasks with well-defined training phases.
Use Plateau Decay for models that train on highly variable datasets.

# RANDOMIZE THE IMAGES USED WITHIN A BATCH, BECAUSE IMAGES ARE RELATED TO ONE OTHER AND MAY IMPACT THE GRADIENT CALCULATIONS.

In [ ]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.optim import SGD
from torchvision import transforms
from tqdm import tqdm
from ultralytics import YOLO

# Define the custom Dataset
class VideoFrameDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        
        # Organize images by video_id
        self.data = {}
        self.all_images = []
        for image_file in os.listdir(image_dir):
            video_id = image_file.split('-')[0]  # Extract video_id from filename
            if video_id not in self.data:
                self.data[video_id] = []
            self.data[video_id].append(image_file)
        
        # Flatten the dataset for indexing
        for video_id, images in self.data.items():
            for image in images:
                self.all_images.append((video_id, image))

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        video_id, image_file = self.all_images[idx]
        image_path = os.path.join(self.image_dir, image_file)
        label_path = os.path.join(self.label_dir, image_file.replace('.jpg', '.txt'))  # Assuming labels are in .txt format
        
        # Load image
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        # Load label
        with open(label_path, 'r') as f:
            label = f.read()  # Modify based on your label format
        
        return image, label, video_id

# Define the BalancedBatchSampler
class BalancedBatchSampler(Sampler):
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        
        # Group indices by video_id
        self.grouped_data = {}
        for idx, (video_id, _) in enumerate(data.all_images):
            if video_id not in self.grouped_data:
                self.grouped_data[video_id] = []
            self.grouped_data[video_id].append(idx)
        
        # Shuffle indices within each video
        for video_id in self.grouped_data:
            random.shuffle(self.grouped_data[video_id])

    def __iter__(self):
        # Generate balanced batches with equal frames from each video
        video_ids = list(self.grouped_data.keys())
        video_pointers = {video_id: 0 for video_id in video_ids}
        batch = []
        
        while True:
            for video_id in video_ids:
                if video_pointers[video_id] < len(self.grouped_data[video_id]):
                    batch.append(self.grouped_data[video_id][video_pointers[video_id]])
                    video_pointers[video_id] += 1
                    
                # When batch is full, yield it
                if len(batch) == self.batch_size:
                    yield batch
                    batch = []
            
            # Stop when all videos are exhausted
            if all(video_pointers[video_id] >= len(self.grouped_data[video_id]) for video_id in video_ids):
                break

    def __len__(self):
        return sum(len(indices) for indices in self.grouped_data.values()) // self.batch_size

# Paths for images and labels
image_dir = "/kaggle/working/images/train"
label_dir = "/kaggle/working/labels/train"

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Adjust normalization values if needed
])

# Create the dataset
dataset = VideoFrameDataset(image_dir=image_dir, label_dir=label_dir, transform=transform)

# Create the sampler and DataLoader
batch_size = 32
sampler = BalancedBatchSampler(dataset, batch_size=batch_size)
dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler, num_workers=4)

# Load the YOLOv8 model
model = YOLO('yolov8m.pt')

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = SGD(model.parameters(), lr=0.002, momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)  # 5 epochs

# Training loop
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    running_loss = 0.0

    for batch in tqdm(dataloader, desc="Training"):
        # Unpack batch
        images, labels, video_ids = batch
        images = images.to(device)

        # Forward pass
        preds = model(images)

        # Compute YOLO loss
        loss = model.loss(preds, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track loss
        running_loss += loss.item()

    # Step the scheduler
    scheduler.step()

    # Print loss for the epoch
    print(f"Epoch [{epoch + 1}/{epochs}] Loss: {running_loss / len(dataloader):.4f}")

# Save the trained model
model.save('custom_yolov8m.pt')

print("Training complete. Model saved as 'custom_yolov8m.pt'.")


# FINAL VERSION


In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # CHANGED
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.001,     # Regularization, increased from 0.0005
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=50             # Epochs to wait for no observable improvement for early stopping of training
)

sgd

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # CHANGED
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    weight_decay=0.001,     # Regularization, increased from 0.0005
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=50             # Epochs to wait for no observable improvement for early stopping of training
)

lrf=0.001

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=5,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # CHANGED
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.001,                # Learning rate factor remains the same
    weight_decay=0.001,     # Regularization, increased from 0.0005
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=50             # Epochs to wait for no observable improvement for early stopping of training
)

AUGMENTATION + ADAMW

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=100,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # CHANGED
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    
    weight_decay=0.001,     # Regularization, increased from 0.0005
    
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=50,             # Epochs to wait for no observable improvement for early stopping of training

    conf=0.3,

    augment=True,                       # Enable basic augmentations
    
    # Augmentation parameters
    hsv_h=0.015,                        # Hue shift (0.0 - 0.5), where 0.015 is a small shift
    hsv_s=0.7,                          # Saturation shift (0.0 - 0.9)
    hsv_v=0.4,                          # Brightness shift (0.0 - 0.9)
    degrees=10.0,                       # Range of rotation in degrees (-180 to 180)
    translate=0.1,                      # Range of translation (0.0 - 1.0), fraction of image size
    scale=0.5,                          # Scale factor range (0.0 - 1.0)
    shear=2.0,                          # Shear angle in degrees
    perspective=0.0001,                 # Perspective distortion (0.0 - 0.001)
    flipud=0.5,                         # Probability of vertical flip (0.0 - 1.0)
    fliplr=0.5,                         # Probability of horizontal flip (0.0 - 1.0)
    mosaic=1.0,                         # Probability of using mosaic augmentation (0.0 - 1.0)
    mixup=0.2,                          # Probability of using mixup augmentation (0.0 - 1.0)
    copy_paste=0.0,                     # Probability of copy-paste augmentation (0.0 - 1.0)
    auto_augment='randaugment',         # Auto augment policy ('randaugment', 'trivialaugment', etc.)
    erasing=0.4,                        # Probability of random erasing (cutout) augmentation
    crop_fraction=1.0 
)

NO AUGMENTATION + SGD

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=100,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='SGD',       # CHANGED
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    
    weight_decay=0.001,     # Regularization, increased from 0.0005
    
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=50,             # Epochs to wait for no observable improvement for early stopping of training

    conf=0.25
)

No Augmentation, ADAMW

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=50,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # CHANGED
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    
    weight_decay=0.001,     # Regularization, increased from 0.0005
    
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=10,             # Epochs to wait for no observable improvement for early stopping of training

    conf=0.25
)

TESTING SMALL WITH THE PARAMETERS

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8s.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=30,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # CHANGED
    lr0=0.007, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    
    weight_decay=0.001,     # Regularization, increased from 0.0005
    
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=10,             # Epochs to wait for no observable improvement for early stopping of training

    conf=0.3,

    augment=True,                       # Enable basic augmentations
    
    # Augmentation parameters
    hsv_h=0.015,                        # Hue shift (0.0 - 0.5), where 0.015 is a small shift
    hsv_s=0.7,                          # Saturation shift (0.0 - 0.9)
    hsv_v=0.4,                          # Brightness shift (0.0 - 0.9)
    degrees=10.0,                       # Range of rotation in degrees (-180 to 180)
    translate=0.1,                      # Range of translation (0.0 - 1.0), fraction of image size
    scale=0.5,                          # Scale factor range (0.0 - 1.0)
    shear=2.0,                          # Shear angle in degrees
    perspective=0.0001,                 # Perspective distortion (0.0 - 0.001)
    flipud=0.5,                         # Probability of vertical flip (0.0 - 1.0)
    fliplr=0.5,                         # Probability of horizontal flip (0.0 - 1.0)
    mosaic=1.0,                         # Probability of using mosaic augmentation (0.0 - 1.0)
    mixup=0.2,                          # Probability of using mixup augmentation (0.0 - 1.0)
    copy_paste=0.0,                     # Probability of copy-paste augmentation (0.0 - 1.0)
    auto_augment='randaugment',         # Auto augment policy ('randaugment', 'trivialaugment', etc.)
    erasing=0.4,                        # Probability of random erasing (cutout) augmentation
    crop_fraction=1.0 
)

Augmentation + SGD

In [ ]:
# Build from YAML and transfer weights
Final_model = YOLO('yolov8m.pt')

# Training The Final Model
Result_Final_model = Final_model.train(
    data="/kaggle/working/data.yaml",
    seed=0,                   # Random seed for reproducibility
    epochs=30,               # Medium model requires fewer epochs; start with 30 but we have patience = 50 thats why keeping it 100 as we don't need to worry
    batch=32,                # Use a larger batch size if memory allows (medium model uses less memory)
    optimizer='AdamW',       # CHANGED
    lr0=0.005, # Higher, earlier 0.002
    lrf=0.01,                # Learning rate factor remains the same
    
    weight_decay=0.001,     # Regularization, increased from 0.0005
    
    warmup_epochs=3,         # Warmup epochs to stabilize initial training
    cos_lr=True,              # Cosine learning rate decay for smoother optimization
    patience=50,             # Epochs to wait for no observable improvement for early stopping of training

    conf=0.25,

    augment=True,                       # Enable basic augmentations
    
    # Augmentation parameters
    hsv_h=0.015,                        # Hue shift (0.0 - 0.5), where 0.015 is a small shift
    hsv_s=0.7,                          # Saturation shift (0.0 - 0.9)
    hsv_v=0.4,                          # Brightness shift (0.0 - 0.9)
    degrees=10.0,                       # Range of rotation in degrees (-180 to 180)
    translate=0.1,                      # Range of translation (0.0 - 1.0), fraction of image size
    scale=0.5,                          # Scale factor range (0.0 - 1.0)
    shear=2.0,                          # Shear angle in degrees
    perspective=0.0001,                 # Perspective distortion (0.0 - 0.001)
    flipud=0.5,                         # Probability of vertical flip (0.0 - 1.0)
    fliplr=0.5,                         # Probability of horizontal flip (0.0 - 1.0)
    mosaic=1.0,                         # Probability of using mosaic augmentation (0.0 - 1.0)
    mixup=0.2,                          # Probability of using mixup augmentation (0.0 - 1.0)
    copy_paste=0.0,                     # Probability of copy-paste augmentation (0.0 - 1.0)
    auto_augment='randaugment',         # Auto augment policy ('randaugment', 'trivialaugment', etc.)
    erasing=0.4,                        # Probability of random erasing (cutout) augmentation
    crop_fraction=1.0 
)

In [ ]:
!nvidia-smi --gpu-reset

In [ ]:
!python val.py --data {yaml_file} --weights runs/detect/train4/weights/best.pt

In [ ]:
# Load the best model weights after training
model = YOLO('runs/detect/train4/weights/best.pt')

In [ ]:
import os

def read_and_print_files(directory, n=None):
    """
    Reads files from a directory and prints their content.

    Args:
        directory (str): The path to the directory.
        n (int, optional): Number of files to process. If None, processes all files.
    """
    # List all files in the directory
    file_list = [os.path.join(directory, file) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]
    
    # Sort files alphabetically (optional, for consistent processing order)
    file_list.sort()
    
    # Process only the first n files if n is specified
    if n is not None:
        file_list = file_list[:n]
    
    for file in file_list:
        print(f"\nReading file: {file}")
        try:
            with open(file, 'r') as f:
                # Read and print each line
                for line in f:
                    # Remove extra spaces and split integers (optional processing)
                    line_content = line.strip()
                    print(line_content)
        except Exception as e:
            print(f"Error reading file {file}: {e}")

# Directory path
workingdir = "/kaggle/working/labels/train"

# Specify the number of files to read (or None for all files)
n_files_to_read = 10

# Call the function
read_and_print_files(workingdir, n_files_to_read if n_files_to_read > 0 else None)


**Sample test with image of choice**

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
id = "1-1008"

# Prediction step
results = model.predict(
    source="/kaggle/working/images/train/"+id+".jpg",
    save=True,
    conf=0.25
)

# Accessing prediction results
result = results[0]

# Plotting the predictions
res_plotted = result.plot()
res_plotted = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)

# Load the original label file
label_path = "/kaggle/working/labels/train/"+id+".txt"
image_path = "/kaggle/working/images/train/"+id+".jpg"

try:
    print(f"Reading original bounding box data from: {label_path}")
    with open(label_path, 'r') as f:
        original_boxes = f.readlines()
    
    # Load the original image
    original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    
    height, width, _ = original_image.shape
    
    # Draw original bounding boxes on the original image
    for box in original_boxes:
        data = list(map(float, box.strip().split()))
        cls, x_center, y_center, w, h = data
        x1 = int((x_center - w / 2) * width)
        y1 = int((y_center - h / 2) * height)
        x2 = int((x_center + w / 2) * width)
        y2 = int((y_center + h / 2) * height)
        
        # Draw rectangle for the original label
        cv2.rectangle(res_plotted, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)  # Blue color for original labels
        
except FileNotFoundError:
    print(f"Error: Label file {label_path} not found.")
except Exception as e:
    print(f"An error occurred while processing the label file: {e}")

# Overlay predictions on top of the original bounding box image
height, width, _ = res_plotted.shape

# Draw predictions again with red color
for box in result.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
    conf = box.conf[0] if box.conf is not None else 0.0
    cls = int(box.cls[0]) if box.cls is not None else -1
    cv2.rectangle(res_plotted, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)  # Red color for predictions
    cv2.putText(
        res_plotted,
        f"{cls} {conf:.2f}",
        (x1, y1 - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (255, 0, 0),
        1,
        cv2.LINE_AA,
    )

# Display the image with predictions (red) and original labels (blue)
plt.figure(figsize=(10, 10))
plt.imshow(res_plotted)
plt.axis("off")
plt.title("Image with Predictions (Red) and Original Labels (Blue)")
plt.show()


# Creating The submission file

In [ ]:
import pandas as pd
import cv2
from PIL import Image
from ultralytics import YOLO

# Load the test CSV file
test_df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/test.csv")

# Load the trained model
model = YOLO('runs/detect/train4/weights/best.pt')

# Define the base directory for images
base_image_dir = "/kaggle/input/tensorflow-great-barrier-reef/train_images/"

# List to hold results for submission
submission_data = []

# Loop through each row in the test DataFrame
for index, row in test_df.iterrows():
    # Construct the image path based on video_id and video_frame
    video_id = row['video_id']
    video_frame = row['video_frame']
    image_path = f"{base_image_dir}video_{video_id-1}/{video_frame}.jpg"
    
    # Run the prediction on the current image
    results = model.predict(source=image_path, conf=0.2, iou=0.5)
    
    # Prepare annotations for the current image
    annotations = []
    for box in results[0].boxes.data:  # Iterate over each detected box
        confidence = float(box[4].item())  # Confidence score
        x_min = float(box[0].item())
        y_min = float(box[1].item())
        x_max = float(box[2].item())
        y_max = float(box[3].item())
        
        # Convert to width and height
        width = x_max - x_min
        height = y_max - y_min
        
        # Format as "confidence x_min y_min width height"
        annotations.append(f"{confidence:.2f} {x_min} {y_min} {width} {height}")
    
    # Join all annotations with a space separator, or leave blank if no detections
    annotation_str = " ".join(annotations) if annotations else ""
    
    # Append result to submission data
    submission_data.append({"row_num": index, "annotations": annotation_str})

# Create DataFrame for submission and save to CSV
submission_df = pd.DataFrame(submission_data)
submission_df.to_csv("submission.csv", index=False)


In [ ]:
!cat submission.csv

In [ ]:
boxes

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (pixel_array, sample_prediction_df) in iter_test:
    sample_prediction_df['annotations'] = '0.5 0 0 100 100'  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions
